#Regression Analysis

In [1]:
import pandas as pd
df=pd.read_excel('All_Project_data_WITH_Amenity_Scores.xlsx')
df

,Project_ID,Project_Name,Village,City,project_lat,Commencement Date,Age_Of_The_Building_Till_11thOct2025,project_lng,No_of_Floor,Carpet_Rate_Lower,...,old_Nearby_Amenities,is_pune,Category_numeric,Cluster_LatLong,Cluster_LatLongRate,Cluster_LatLongCategory,Avg rate of village on Net Carpet Area,Upper and lower avg,diff,amenity_score
0,3536,Kk Ankur,Akurdi,Pune,18.648752,2017-08-06,8.268493,73.779809,5,5184.040138,...,"[{'Name': 'Dalvi Nagar Bus Stop', 'Amenity typ...",True,1.0,740.0,28.0,157.0,8219.116197,5511.251887,2707.864310,0.646544
1,4270,Mayur Samruddhi,Akurdi,Pune,18.650024,2017-08-07,8.265753,73.780231,7,7409.040000,...,"[{'Name': 'Dalvi Nagar Bus Stop', 'Amenity typ...",True,2.0,740.0,778.0,1.0,8219.116197,7993.635000,225.481197,0.634250
2,3803,Laxman Plaza,Akurdi,Pune,18.654797,2017-08-10,8.257534,73.777009,7,6239.056251,...,"[{'Name': 'Nigdi Bus Stand', 'Amenity type': '...",True,1.0,783.0,460.0,687.0,8219.116197,6575.300874,1643.815323,0.642940
3,939,Arihant Heights,Akurdi,Pune,18.658221,2017-08-11,8.254795,73.766842,6,9631.323261,...,"[{'Name': 'Gurudwara Bus Stop', 'Amenity type'...",True,1.0,576.0,734.0,277.0,8219.116197,10521.845880,2302.729682,0.613469
4,6768,Shantai Heritage,Akurdi,Pune,18.648880,2017-08-17,8.238356,73.778450,6,9833.295000,...,"[{'Name': 'Dalvi Nagar Bus Stop', 'Amenity typ...",True,1.0,740.0,87.0,157.0,8219.116197,9992.227500,1773.111303,0.614370
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
33279,Nr32993,Chandrabhaga Residency,Yewalewadi,Pune,18.454105,NaT,NaN,73.850453,0,2791.563386,...,"[{'Name': 'Bharati Vidyapeeth Bus Stop', 'Amen...",True,1.0,754.0,463.0,303.0,5760.499668,3077.414659,2683.085009,0.534407
33280,Nr33020,Rehman Apartment,Yewalewadi,Pune,18.435904,NaT,NaN,73.891445,0,2781.148806,...,"[{'Name': 'Yewalewadi', 'Amenity type': 'Bus_s...",True,1.0,68.0,760.0,103.0,5760.499668,2985.126032,2775.373637,0.165188
33281,Nr33039,Unity Heights,Yewalewadi,Pune,18.472783,NaT,NaN,73.863211,0,2813.432933,...,"[{'Name': 'Indira Nagar Bus Stop', 'Amenity ty...",True,2.0,626.0,590.0,22.0,5760.499668,2889.201418,2871.298250,0.525973
33282,Nr33011,Nk Heights,Yewalewadi,Pune,18.435031,NaT,NaN,73.889648,0,2755.460938,...,"[{'Name': 'Yewalewadi', 'Amenity type': 'Bus_s...",True,1.0,68.0,760.0,103.0,5760.499668,2864.854253,2895.645415,0.128698


In [2]:
df.columns

Index(['Project_ID', 'Project_Name', 'Village', 'City', 'project_lat',
       'Commencement Date', 'Age_Of_The_Building_Till_11thOct2025',
       'project_lng', 'No_of_Floor', 'Carpet_Rate_Lower', 'Carpet_Rate_Upper',
       'Saleable_Rate_Lower', 'Saleable_Rate_Upper', 'Mid_Rate',
       'Builtup_Rate_Lower', 'Builtup_Rate_Upper', 'adjoining_highway',
       'all_roads_in_200m', 'Highway Type', 'Road_Category',
       'old_Contribution_Score', 'old_Amenity_Category',
       'old_Nearby_Amenities', 'is_pune', 'Category_numeric',
       'Cluster_LatLong', 'Cluster_LatLongRate', 'Cluster_LatLongCategory',
       'Avg rate of village on Net Carpet Area', 'Upper and lower avg', 'diff',
       'amenity_score'],
      dtype='object')

In [4]:
import pandas as pd
from sklearn.linear_model import LinearRegression
import numpy as np
import os

# Assuming your dataframe is named df
# df = pd.read_csv('your_data.csv')  # Uncomment and adjust if needed

# Function to perform regressions for a given cluster column
def perform_regressions(df, cluster_col, regression_types=['amenity', 'roadcat', 'both']):
    # Drop rows without cluster or Category_numeric for this specific column and create a copy
    df_filtered = df.dropna(subset=[cluster_col, 'Category_numeric']).copy()
    
    # Print unique Category_numeric values for debugging
    unique_categories = df_filtered['Category_numeric'].unique()
    print(f"Unique Category_numeric values in {cluster_col}: {unique_categories}")
    
    # Verify Category_numeric is numerical (only needed if regression includes Category_numeric)
    if 'roadcat' in regression_types or 'both' in regression_types:
        if not np.issubdtype(df_filtered['Category_numeric'].dtype, np.number):
            print(f"Error: Category_numeric in {cluster_col} contains non-numerical values: {unique_categories}")
            raise ValueError(f"Category_numeric in {cluster_col} must be numerical.")
    
    # Group by cluster
    groups = df_filtered.groupby(cluster_col)
    
    # Prepare results lists for each regression type
    results_amenity = []
    results_roadcat = []
    results_both = []
    
    for cluster, group in groups:
        n = len(group)
        
        # Regression 1: X = amenity_score, y = Mid_Rate  ✅ CHANGED
        if 'amenity' in regression_types:
            if n >= 2:
                try:
                    X = group[['amenity_score']]  # ✅ CHANGED FROM Contribution_Score
                    y = group['Mid_Rate']
                    model = LinearRegression().fit(X, y)
                    slope = model.coef_[0]
                    intercept = model.intercept_
                    equation = f"y = {slope:.4f} * x + {intercept:.4f}"
                    results_amenity.append({
                        'Cluster': cluster,
                        'Equation': equation,
                        'Slope_Amenity': slope,  # ✅ CHANGED
                        'Intercept': intercept,
                        'Num_Projects': n,
                        'Note': ''
                    })
                except Exception as e:
                    results_amenity.append({
                        'Cluster': cluster,
                        'Equation': '',
                        'Slope_Amenity': '',  # ✅ CHANGED
                        'Intercept': '',
                        'Num_Projects': n,
                        'Note': f"Error: {str(e)}"
                    })
            else:
                results_amenity.append({
                    'Cluster': cluster,
                    'Equation': '',
                    'Slope_Amenity': '',  # ✅ CHANGED
                    'Intercept': '',
                    'Num_Projects': n,
                    'Note': 'Insufficient samples (need at least 2)'
                })
        
        # Regression 2: X = Category_numeric, y = Mid_Rate (UNCHANGED)
        if 'roadcat' in regression_types:
            if n >= 2:
                try:
                    X = group[['Category_numeric']]
                    y = group['Mid_Rate']
                    model = LinearRegression().fit(X, y)
                    slope = model.coef_[0]
                    intercept = model.intercept_
                    equation = f"y = {slope:.4f} * x + {intercept:.4f}"
                    results_roadcat.append({
                        'Cluster': cluster,
                        'Equation': equation,
                        'Slope_RoadCat': slope,
                        'Intercept': intercept,
                        'Num_Projects': n,
                        'Note': ''
                    })
                except Exception as e:
                    results_roadcat.append({
                        'Cluster': cluster,
                        'Equation': '',
                        'Slope_RoadCat': '',
                        'Intercept': '',
                        'Num_Projects': n,
                        'Note': f"Error: {str(e)}"
                    })
            else:
                results_roadcat.append({
                    'Cluster': cluster,
                    'Equation': '',
                    'Slope_RoadCat': '',
                    'Intercept': '',
                    'Num_Projects': n,
                    'Note': 'Insufficient samples (need at least 2)'
                })
        
        # Regression 3: X1 = amenity_score, X2 = Category_numeric, y = Mid_Rate  ✅ CHANGED
        if 'both' in regression_types:
            if n >= 3:
                try:
                    X = group[['amenity_score', 'Category_numeric']]  # ✅ CHANGED
                    y = group['Mid_Rate']
                    model = LinearRegression().fit(X, y)
                    coef_amenity = model.coef_[0]  # ✅ CHANGED
                    coef_roadcat = model.coef_[1]
                    intercept = model.intercept_
                    equation = f"y = {coef_amenity:.4f} * x1 + {coef_roadcat:.4f} * x2 + {intercept:.4f}"
                    results_both.append({
                        'Cluster': cluster,
                        'Equation': equation,
                        'Slope_Amenity': coef_amenity,  # ✅ CHANGED
                        'Slope_RoadCat': coef_roadcat,
                        'Intercept': intercept,
                        'Num_Projects': n,
                        'Note': ''
                    })
                except Exception as e:
                    results_both.append({
                        'Cluster': cluster,
                        'Equation': '',
                        'Slope_Amenity': '',  # ✅ CHANGED
                        'Slope_RoadCat': '',
                        'Intercept': '',
                        'Num_Projects': n,
                        'Note': f"Error: {str(e)}"
                    })
            else:
                results_both.append({
                    'Cluster': cluster,
                    'Equation': '',
                    'Slope_Amenity': '',  # ✅ CHANGED
                    'Slope_RoadCat': '',
                    'Intercept': '',
                    'Num_Projects': n,
                    'Note': 'Insufficient samples (need at least 3)'
                })
    
    return pd.DataFrame(results_amenity), pd.DataFrame(results_roadcat), pd.DataFrame(results_both)

# Perform regressions for Cluster_LatLong (all three regressions)
try:
    df_res_amenity_ll, df_res_roadcat_ll, df_res_both_ll = perform_regressions(
        df, 'Cluster_LatLong', regression_types=['amenity', 'roadcat', 'both']  # ✅ CHANGED
    )
except ValueError as e:
    print(e)
    print("Stopping execution due to issues in Cluster_LatLong.")
    raise

# Perform regressions for Cluster_LatLongRate (all three regressions)
try:
    df_res_amenity_llr, df_res_roadcat_llr, df_res_both_llr = perform_regressions(
        df, 'Cluster_LatLongRate', regression_types=['amenity', 'roadcat', 'both']  # ✅ CHANGED
    )
except ValueError as e:
    print(e)
    print("Stopping execution due to issues in Cluster_LatLongRate.")
    raise

# Perform regression for Cluster_LatLongCategory (only amenity_score vs Mid_Rate)  ✅ CHANGED
try:
    df_res_amenity_llc, _, _ = perform_regressions(
        df, 'Cluster_LatLongCategory', regression_types=['amenity']  # ✅ CHANGED
    )
except ValueError as e:
    print(e)
    print("Stopping execution due to issues in Cluster_LatLongCategory.")
    raise

# Write to Excel file with seven sheets with clear, simple names
excel_file = 'regression_results.xlsx'
with pd.ExcelWriter(excel_file, engine='openpyxl', mode='w') as writer:
    df_res_amenity_ll.to_excel(writer, sheet_name='LatLong_Amenity_vs_Rate', index=False)  # ✅ CHANGED
    df_res_roadcat_ll.to_excel(writer, sheet_name='LatLong_RoadCat_vs_Rate', index=False)
    df_res_both_ll.to_excel(writer, sheet_name='LatLong_Both_vs_Rate', index=False)
    df_res_amenity_llr.to_excel(writer, sheet_name='LatLongRate_Amenity_vs_Rate', index=False)  # ✅ CHANGED
    df_res_roadcat_llr.to_excel(writer, sheet_name='LatLongRate_RoadCat_vs_Rate', index=False)
    df_res_both_llr.to_excel(writer, sheet_name='LatLongRate_Both_vs_Rate', index=False)
    df_res_amenity_llc.to_excel(writer, sheet_name='LatLongCategory_Amenity_vs_Rate', index=False)  # ✅ CHANGED

print(f"Results written to {excel_file}")

Unique Category_numeric values in Cluster_LatLong: [1. 2. 3. 4.]
Unique Category_numeric values in Cluster_LatLongRate: [1. 2. 3. 4.]
Unique Category_numeric values in Cluster_LatLongCategory: [1. 2. 3. 4.]
Results written to regression_results.xlsx


In [6]:
import pandas as pd
from sklearn.linear_model import LinearRegression
import numpy as np
import os

# Assuming your dataframe is named df
# df = pd.read_csv('your_data.csv')  # Uncomment and adjust if needed

# 🚀 NEW: DROP DUPLICATES FIRST
print(f"Original dataset: {len(df)} rows")
df = df.drop_duplicates(subset=['project_lat', 'project_lng'], keep='first')
print(f"After removing duplicates: {len(df)} rows")
print("---")

# Function to perform regressions for a given cluster column
def perform_regressions(df, cluster_col, regression_types=['amenity', 'roadcat', 'both']):
    # Drop rows without cluster or Category_numeric for this specific column and create a copy
    df_filtered = df.dropna(subset=[cluster_col, 'Category_numeric']).copy()
    
    # Print unique Category_numeric values for debugging
    unique_categories = df_filtered['Category_numeric'].unique()
    print(f"Unique Category_numeric values in {cluster_col}: {unique_categories}")
    
    # Verify Category_numeric is numerical (only needed if regression includes Category_numeric)
    if 'roadcat' in regression_types or 'both' in regression_types:
        if not np.issubdtype(df_filtered['Category_numeric'].dtype, np.number):
            print(f"Error: Category_numeric in {cluster_col} contains non-numerical values: {unique_categories}")
            raise ValueError(f"Category_numeric in {cluster_col} must be numerical.")
    
    # Group by cluster
    groups = df_filtered.groupby(cluster_col)
    
    # Prepare results lists for each regression type
    results_amenity = []
    results_roadcat = []
    results_both = []
    
    for cluster, group in groups:
        n = len(group)
        
        # Regression 1: X = amenity_score, y = Mid_Rate
        if 'amenity' in regression_types:
            if n >= 2:
                try:
                    X = group[['amenity_score']]
                    y = group['Mid_Rate']
                    model = LinearRegression().fit(X, y)
                    slope = model.coef_[0]
                    intercept = model.intercept_
                    equation = f"y = {slope:.4f} * x + {intercept:.4f}"
                    results_amenity.append({
                        'Cluster': cluster,
                        'Equation': equation,
                        'Slope_Amenity': slope,
                        'Intercept': intercept,
                        'Num_Projects': n,
                        'Note': ''
                    })
                except Exception as e:
                    results_amenity.append({
                        'Cluster': cluster,
                        'Equation': '',
                        'Slope_Amenity': '',
                        'Intercept': '',
                        'Num_Projects': n,
                        'Note': f"Error: {str(e)}"
                    })
            else:
                results_amenity.append({
                    'Cluster': cluster,
                    'Equation': '',
                    'Slope_Amenity': '',
                    'Intercept': '',
                    'Num_Projects': n,
                    'Note': 'Insufficient samples (need at least 2)'
                })
        
        # Regression 2: X = Category_numeric, y = Mid_Rate (UNCHANGED)
        if 'roadcat' in regression_types:
            if n >= 2:
                try:
                    X = group[['Category_numeric']]
                    y = group['Mid_Rate']
                    model = LinearRegression().fit(X, y)
                    slope = model.coef_[0]
                    intercept = model.intercept_
                    equation = f"y = {slope:.4f} * x + {intercept:.4f}"
                    results_roadcat.append({
                        'Cluster': cluster,
                        'Equation': equation,
                        'Slope_RoadCat': slope,
                        'Intercept': intercept,
                        'Num_Projects': n,
                        'Note': ''
                    })
                except Exception as e:
                    results_roadcat.append({
                        'Cluster': cluster,
                        'Equation': '',
                        'Slope_RoadCat': '',
                        'Intercept': '',
                        'Num_Projects': n,
                        'Note': f"Error: {str(e)}"
                    })
            else:
                results_roadcat.append({
                    'Cluster': cluster,
                    'Equation': '',
                    'Slope_RoadCat': '',
                    'Intercept': '',
                    'Num_Projects': n,
                    'Note': 'Insufficient samples (need at least 2)'
                })
        
        # Regression 3: X1 = amenity_score, X2 = Category_numeric, y = Mid_Rate
        if 'both' in regression_types:
            if n >= 3:
                try:
                    X = group[['amenity_score', 'Category_numeric']]
                    y = group['Mid_Rate']
                    model = LinearRegression().fit(X, y)
                    coef_amenity = model.coef_[0]
                    coef_roadcat = model.coef_[1]
                    intercept = model.intercept_
                    equation = f"y = {coef_amenity:.4f} * x1 + {coef_roadcat:.4f} * x2 + {intercept:.4f}"
                    results_both.append({
                        'Cluster': cluster,
                        'Equation': equation,
                        'Slope_Amenity': coef_amenity,
                        'Slope_RoadCat': coef_roadcat,
                        'Intercept': intercept,
                        'Num_Projects': n,
                        'Note': ''
                    })
                except Exception as e:
                    results_both.append({
                        'Cluster': cluster,
                        'Equation': '',
                        'Slope_Amenity': '',
                        'Slope_RoadCat': '',
                        'Intercept': '',
                        'Num_Projects': n,
                        'Note': f"Error: {str(e)}"
                    })
            else:
                results_both.append({
                    'Cluster': cluster,
                    'Equation': '',
                    'Slope_Amenity': '',
                    'Slope_RoadCat': '',
                    'Intercept': '',
                    'Num_Projects': n,
                    'Note': 'Insufficient samples (need at least 3)'
                })
    
    return pd.DataFrame(results_amenity), pd.DataFrame(results_roadcat), pd.DataFrame(results_both)

# Perform regressions for Cluster_LatLong (all three regressions)
try:
    df_res_amenity_ll, df_res_roadcat_ll, df_res_both_ll = perform_regressions(
        df, 'Cluster_LatLong', regression_types=['amenity', 'roadcat', 'both']
    )
except ValueError as e:
    print(e)
    print("Stopping execution due to issues in Cluster_LatLong.")
    raise

# Perform regressions for Cluster_LatLongRate (all three regressions)
try:
    df_res_amenity_llr, df_res_roadcat_llr, df_res_both_llr = perform_regressions(
        df, 'Cluster_LatLongRate', regression_types=['amenity', 'roadcat', 'both']
    )
except ValueError as e:
    print(e)
    print("Stopping execution due to issues in Cluster_LatLongRate.")
    raise

# Perform regression for Cluster_LatLongCategory (only amenity_score vs Mid_Rate)
try:
    df_res_amenity_llc, _, _ = perform_regressions(
        df, 'Cluster_LatLongCategory', regression_types=['amenity']
    )
except ValueError as e:
    print(e)
    print("Stopping execution due to issues in Cluster_LatLongCategory.")
    raise

# Write to Excel file with seven sheets with clear, simple names
excel_file = 'regression_results2.xlsx'
with pd.ExcelWriter(excel_file, engine='openpyxl', mode='w') as writer:
    df_res_amenity_ll.to_excel(writer, sheet_name='LatLong_Amenity_vs_Rate', index=False)
    df_res_roadcat_ll.to_excel(writer, sheet_name='LatLong_RoadCat_vs_Rate', index=False)
    df_res_both_ll.to_excel(writer, sheet_name='LatLong_Both_vs_Rate', index=False)
    df_res_amenity_llr.to_excel(writer, sheet_name='LatLongRate_Amenity_vs_Rate', index=False)
    df_res_roadcat_llr.to_excel(writer, sheet_name='LatLongRate_RoadCat_vs_Rate', index=False)
    df_res_both_llr.to_excel(writer, sheet_name='LatLongRate_Both_vs_Rate', index=False)
    df_res_amenity_llc.to_excel(writer, sheet_name='LatLongCategory_Amenity_vs_Rate', index=False)

print(f"Results written to {excel_file}")

Original dataset: 26675 rows
After removing duplicates: 26675 rows
---
Unique Category_numeric values in Cluster_LatLong: [1. 2. 3. 4.]
Unique Category_numeric values in Cluster_LatLongRate: [1. 2. 3. 4.]
Unique Category_numeric values in Cluster_LatLongCategory: [1. 2. 3. 4.]
Results written to regression_results2.xlsx
